In [100]:
import string
import os
import re
import pandas as pd

In [101]:
pd.set_option('display.max_rows', None)
os.chdir(r"C:\Users\tmitc\notebooks\mini-project\diary")
year_1804 = []
year_1805 = []

all_entries = os.listdir()

In [102]:
for f in all_entries:
    if f.startswith('1804'):
        entry = open(f, "r")
        year_1804.append(entry.read().lower().replace('#', '').replace('$', ''))
    elif f.startswith('1805'):
        entry = open(f, "r")
        year_1805.append(entry.read().lower().replace('#', '').replace('$', ''))

In [103]:
print(year_1804)

['cloudy, snowd at night. mr ballard and ephraim to meeting. i have been unwell. son jonathan, his wife and 6 children supt here. we had a puding and roast spare rib. i was very unwell all night but, as is usual, did with out much care taken of me. rachel to bed at 8 oclock. at home, very unwell.', 'clear. mr ballard is gone to ye meddow. i feel very unwel. my husband came home at 4h pm, took a little food. complains of feeling the pain in his stomach, but was calld by john sewall to answer an execution of 800 dollars. was by him conducted to the jail in augusta and commited. our two son*s, jonathan and ephraim were bound for the liberty of the yard for him. i pray the great parent of the universe to protect him and giv him comfort in his present tryal. at home, very unwel. my husband was this evening committdto jail on acount of the tax collection.', 'snowd. son lambard conducted his wife and babe here, left them with me. they tarrie all night, slept with me. i had a very ill turn in 

In [104]:
entries_df = pd.DataFrame(year_1804, columns=["Text"])
entries_df

,Text
0,"cloudy, snowd at night. mr ballard and ephraim..."
1,clear. mr ballard is gone to ye meddow. i feel...
2,snowd. son lambard conducted his wife and babe...
3,cloudy. son lambard came and conducted his wif...
4,clear and cold. son pollard brot his wife & tw...
5,clear. daughter ballard washt here for her sel...
6,clear and cool. i have felt so well as to help...
7,clear. rachel town to meeting afternoon and di...
8,snow storm. son pollard came with a covered sl...
9,cloudy. haman burr conducted rachel town from ...


In [105]:
entries_df['Clear'] = entries_df.Text.str.contains('clear')
entries_df['Cloudy'] = entries_df.Text.str.contains('cloud')
entries_df['Snow'] = entries_df.Text.str.contains('snow')
entries_df['Cold'] = entries_df.Text.str.contains('|'.join(['cold', 'chill', 'snow']))
entries_df['Windy'] = entries_df.Text.str.contains('|'.join(['wind', 'blowd', 'gust', 'blowed']))
entries_df['Sick Count'] = entries_df.Text.str.count('sick') + entries_df.Text.str.count('unwel') + entries_df.Text.str.count('ill')
entries_df

,Text,Clear,Cloudy,Snow,Cold,Windy,Sick Count
0,"cloudy, snowd at night. mr ballard and ephraim...",False,True,True,True,False,3
1,clear. mr ballard is gone to ye meddow. i feel...,True,False,False,False,True,2
2,snowd. son lambard conducted his wife and babe...,False,False,True,True,False,2
3,cloudy. son lambard came and conducted his wif...,False,True,False,False,False,1
4,clear and cold. son pollard brot his wife & tw...,True,False,False,True,False,2
5,clear. daughter ballard washt here for her sel...,True,False,False,False,False,0
6,clear and cool. i have felt so well as to help...,True,False,False,False,False,0
7,clear. rachel town to meeting afternoon and di...,True,False,False,False,False,0
8,snow storm. son pollard came with a covered sl...,False,False,True,True,False,0
9,cloudy. haman burr conducted rachel town from ...,False,True,False,False,False,2
